In [1]:
import sys
import os

# Get the absolute path to the project directory
project_dir = os.path.abspath("..")

# Append the project directory to sys.path
if project_dir not in sys.path:
    sys.path.append(project_dir)

In [2]:
import sys
import os

# Get the absolute path to the project directory
project_dir = os.path.abspath("..")

# Append the project directory to sys.path
if project_dir not in sys.path:
    sys.path.append(project_dir)

import yfinance as yf
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import msgpack
import warnings
from dataclasses import asdict, is_dataclass, dataclass
import os
import msgpack
import json
import pickle
from pandas.api.types import is_datetime64_any_dtype
import scipy
import numpy as np
from scipy.stats import linregress

from src.common.AssetData import AssetData
from src.common.AssetDataService import AssetDataService
from src.common.AssetFileInOut import AssetFileInOut 
from src.databaseService.OutsourceLoader import OutsourceLoader

from src.simulation.SimulatePortfolio import SimulatePortfolio
from src.strategy.StratBuyAndHold import StratBuyAndHold
from src.simulation.ResultAnalyzer import ResultAnalyzer
from src.common.AssetFileInOut import AssetFileInOut
from src.mathTools.CurveAnalysis import CurveAnalysis
from src.predictionModule.FourierML import FourierML
from src.common.YamlTickerInOut import YamlTickerInOut
from src.common.DataFrameTimeOperations import DataFrameTimeOperationsPandas as DFTO


In [3]:
from scipy.special import ellipk, ellipe
from keras.datasets import mnist

A = np.array([
    [1 + 2j, 3 + 4j],
    [5 + 6j, 7 + 8j]
])
B=np.abs(A)
np.minimum(B,2)

array([[2., 2.],
       [2., 2.]])

In [4]:
import numpy as np
from tensorflow.keras.datasets import mnist
from scipy.special import ellipk, ellipe
from scipy.linalg import eig
from numpy import pi, sin, sqrt, arccos, minimum

# Load MNIST data
(X_train, Y_train), (X_test, Y_test) = mnist.load_data()

# Select first samples for training
toSelectTrain=5000
toSelectTest=1000
X_train_trunc = X_train[:toSelectTrain].copy()
Y_train_trunc = Y_train[:toSelectTrain].copy()

X_test_trunc = X_test[:toSelectTest].copy()
Y_test_trunc = Y_test[:toSelectTest].copy()

# Display the shapes
print("XTrain shape:", X_train_trunc.shape)  # Expected: (toSelect, 28, 28)
print("YTrain shape:", Y_train_trunc.shape)  # Expected: (toSelect,)
print("XTest shape:", X_test_trunc.shape)    # Expected: (toSelect, 28, 28)
print(np.max(X_train_trunc))
# One-hot encode YTrain and YTest
num_classes = 10
Ytrain = np.zeros((toSelectTrain, num_classes))
Ytest = np.zeros((toSelectTest, num_classes))
labels = np.arange(10)

# Vectorized one-hot encoding
Ytrain[np.arange(toSelectTrain), Y_train_trunc] = 1
Ytest[np.arange(toSelectTest), Y_test_trunc] = 1

# Reshape images to vectors
U = (X_train_trunc.reshape((toSelectTrain, 28*28)) / np.max(X_train_trunc)).astype("float32")
Utest = (X_test_trunc.reshape((toSelectTest, 28*28)) / np.max(X_test_trunc)).astype("float32")

Util = Utest

XTrain shape: (5000, 28, 28)
YTrain shape: (5000,)
XTest shape: (1000, 28, 28)
255


In [5]:
NRnd = 2000
W = (np.random.randn(NRnd, 28*28)+1j*np.random.randn(NRnd, 28*28))/(sqrt(NRnd))

XMat = (np.abs(W @ U.T)).T
XtilMat = (np.abs(W @ Util.T)).T

K = XMat.T @ XMat
rhs = XMat.T @ Ytrain

C:\Users\KILightTouch\AppData\Local\Temp\ipykernel_16620\1139694220.py:7: RuntimeWarning: invalid value encountered in matmul
  K = XMat.T @ XMat


In [6]:
eigK = np.linalg.eigvalsh(K)
eigK = np.sort(eigK.real)
print(eigK[0:5])

[1.21716328 1.2624484  1.27721516 1.30231756 1.30923146]


In [7]:
KNorm = np.linalg.norm(K, 1)

print(KNorm ** (1/4))

36.01254171055633


In [8]:
g = 30
Ytil = XtilMat @ np.linalg.solve((K + g*np.eye(NRnd)), rhs)

Y_pred = np.argmax(Ytil, axis=1)

# Actual class labels
Y_true = Y_test_trunc

# Calculate accuracy
accuracy = np.mean(Y_pred == Y_true)

print("Accuracy:", accuracy)

Accuracy: 0.93


In [9]:
MaxN=30
max_iter = 10 

low, high = 0.01, MaxN

#low
Ytil = XtilMat @ np.linalg.solve(K + low * np.eye(K.shape[0]), rhs)
acc_low = np.mean(np.argmax(Ytil, axis=1) == Y_true)
acc_low_start=acc_low
#high
Ytil = XtilMat @ np.linalg.solve(K + high * np.eye(K.shape[0]), rhs)
acc_high = np.mean(np.argmax(Ytil, axis=1) == Y_true)
acc_high_start=acc_high

best_acc = acc_low if acc_low>acc_high else acc_high
best_g = low if acc_low>acc_high else high
for i in range(max_iter):
    g = (low + high) / 2
    Ytil = XtilMat @ np.linalg.solve(K + g * np.eye(K.shape[0]), rhs)
    acc = np.mean(np.argmax(Ytil, axis=1) == Y_true)
    if (acc < acc_low and acc < acc_high) or (acc > acc_low and acc > acc_high):
        if acc_low > acc_high:
            best_acc, best_g = acc_low, low
            high=g
            acc_high = acc
        elif acc_low < acc_high:
            best_acc, best_g = acc_high, high
            low=g
            acc_low = acc
    elif acc > acc_low and acc < acc_high:
        best_acc, best_g = acc_high, high
        low=g
        acc_low = acc
    elif acc < acc_low and acc > acc_high:
        best_acc, best_g = acc_low, low
        high=g
        acc_high = acc
    
    elif acc == acc_low and acc > acc_high:
        best_acc, best_g = acc_low, low
        high=g
        acc_high = acc
    elif acc == acc_low and acc < acc_high:
        best_acc, best_g = acc_high, high
        low=g
        acc_low = acc
    elif acc == acc_high and acc > acc_low:
        best_acc, best_g = acc_high, high
        low=g
        acc_low = acc
    elif acc == acc_high and acc < acc_low:
        best_acc, best_g = acc_low, low
        high=g
        acc_high = acc

    elif acc == acc_high and acc == acc_low:
        if acc_low_start > acc_high_start:
            best_acc, best_g = acc_low, low
            high=g
            acc_high = acc
        if acc_high_start > acc_low_start:
            best_acc, best_g = acc_high, high
            low=g
            acc_low = acc

    print(f"acc: {best_acc}   g = {best_g}  low = {low}  high = {high}")

print(best_acc)
print(best_g)

acc: 0.93   g = 30  low = 15.005  high = 30
acc: 0.93   g = 30  low = 15.005  high = 30
acc: 0.93   g = 30  low = 15.005  high = 30
acc: 0.93   g = 30  low = 15.005  high = 30
acc: 0.93   g = 30  low = 15.005  high = 30
acc: 0.93   g = 30  low = 15.005  high = 30
acc: 0.93   g = 30  low = 15.005  high = 30
acc: 0.93   g = 30  low = 15.005  high = 30
acc: 0.93   g = 30  low = 15.005  high = 30
acc: 0.93   g = 30  low = 15.005  high = 30
0.93
30


In [10]:
MaxN = 47
max_iter = 15
low, high = 0.01, MaxN

def compute_acc(g):
    Ytil = XtilMat @ np.linalg.solve(K + g * np.eye(K.shape[0]), rhs)
    return np.mean(np.argmax(Ytil, axis=1) == Y_true)

acc_low = compute_acc(low)
acc_high = compute_acc(high)
best_acc, best_g = (acc_low, low) if acc_low > acc_high else (acc_high, high)

for i in range(max_iter):
    g = (low + high) / 2
    acc = compute_acc(g)
    if acc > best_acc:
        best_acc, best_g = acc, g

    if (acc > acc_low) != (acc > acc_high):
        # If acc is greater than one bound but not the other
        if acc > acc_low:
            low, acc_low = g, acc
        else:
            high, acc_high = g, acc
    else:
        if acc_low > acc_high:
            high, acc_high = g, acc
        else:
            low, acc_low = g, acc

    print(f"acc: {best_acc}   g = {best_g}  low = {low}  high = {high}")

print(best_acc)
print(best_g)


acc: 0.93   g = 23.505  low = 23.505  high = 47
acc: 0.93   g = 23.505  low = 23.505  high = 35.2525
acc: 0.93   g = 23.505  low = 29.378749999999997  high = 35.2525
acc: 0.93   g = 23.505  low = 32.315625  high = 35.2525
acc: 0.93   g = 23.505  low = 33.7840625  high = 35.2525
acc: 0.93   g = 23.505  low = 34.51828125  high = 35.2525
acc: 0.93   g = 23.505  low = 34.885390625  high = 35.2525
acc: 0.93   g = 23.505  low = 35.0689453125  high = 35.2525
acc: 0.93   g = 23.505  low = 35.16072265625  high = 35.2525
acc: 0.93   g = 23.505  low = 35.206611328125  high = 35.2525
acc: 0.93   g = 23.505  low = 35.2295556640625  high = 35.2525
acc: 0.93   g = 23.505  low = 35.24102783203125  high = 35.2525
acc: 0.93   g = 23.505  low = 35.246763916015624  high = 35.2525
acc: 0.93   g = 23.505  low = 35.24963195800781  high = 35.2525
acc: 0.93   g = 23.505  low = 35.251065979003904  high = 35.2525
0.93
23.505
